In [ ]:
import os
import cv2
import glob
import numpy as np
import pandas as pd

# seed
np.random.seed(2018)
from keras.callbacks import Callback, EarlyStopping
from keras.optimizers import *
from keras.callbacks import *
from keras.utils import multi_gpu_model
import sys
sys.path.append('../')
from model.coremodel import model_v9 as my_model
from dataload.my_image_balance_swap import MyImageDataGenerator as MyImageDataGenerator_ba
from dataload.my_image import MyImageDataGenerator
from keras.models import load_model
from model.DepthwiseConv2D import DepthwiseConv2D
print("import done")

In [ ]:
basedir = "/home/ubuntu/work/data"
basemodel_name = 'inception_v3'
model_image_size = (384, 512)
modelname = basemodel_name + '_model-v9'+ '_plan3_20180424'
savemodel_path = "./weights/"

In [ ]:
# define prerpocess_input
from keras.applications.xception import preprocess_input as preprocess_input_ception
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet
stamp = time.strftime('%m%d%H', time.localtime(time.time()))
if basemodel_name == 'resnet50':
    preprocess_input = preprocess_input_resnet
elif basemodel_name == 'xception' or 'inception_v3' or 'inception_resnet_v2':
    preprocess_input = preprocess_input_ception
else:
    print('basemodel_name not defined！')
    raise NameError

modelname =basemodel_name+'-v9-no-weight'+"{}".format(stamp)
train_batch_size = 32
valid_batch_size = 32

In [ ]:
train_gen = MyImageDataGenerator_ba(
    preprocessing_function=preprocess_input,
    width_shift_range=0.05,
    height_shift_range=0.05,
    rotation_range=6,
    shear_range=5,
    channel_shift_range=15,
    zoom_range=(.9, 1.1),
)

gen = MyImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_gen.myflow_from_directory(os.path.join(basedir, 'train'),  model_image_size,
                                                class_mode='binary', shuffle=True, batch_size=train_batch_size)

valid_generator = gen.myflow_from_directory(os.path.join(basedir, 'valid'),  model_image_size,
                                          class_mode='binary', shuffle=True, batch_size=valid_batch_size)
train_samples_epoch = train_generator.samples
print("samples_train_epoch = {}".format(train_samples_epoch))

steps_train = int(len(train_generator)*0.5*3/2)
print("steps_train = {}".format(steps_train))

valid_samples_epoch = valid_generator.samples
print("samples_valid_epoch = {}".format(valid_samples_epoch))

steps_valid = len(valid_generator)
print("steps_valid = {}".format(steps_valid))

In [ ]:
model = my_model(basemodel_name, model_image_size)
multi_model = multi_gpu_model(model, gpus=2)

In [ ]:
class MyModelCheckpoint(Callback):

    def __init__(self, model, path, init_loss=None):
        self.mymodel = model
        self.path = path
        self.best_loss = np.inf
        self.best_acc = -1
      

    def on_epoch_end(self, epoch, logs=None):
        val_acc = logs['val_acc']
        model_name = '{}_{:.4f}{}'.format(modelname, val_acc, '.h5')
        model_path = os.path.join(self.path, model_name)

        if val_acc > self.best_acc:
            print("\nValidation acc increased from {} to {}, saving model to {}".format(self.best_acc, val_acc,
                                                                                        model_path))
            self.mymodel.save_weights(model_path, overwrite=True)
            self.best_acc = val_acc  

In [ ]:
checkpoint = MyModelCheckpoint(model, savemodel_path)
LR_reducer = ReduceLROnPlateau(monitor='val_acc',  mode='max', factor=0.1, patience=2, verbose=1, min_lr=1e-5) 

In [ ]:
multi_model.compile(optimizer=Adam(lr=1e-3), 
              loss='binary_crossentropy',
              metrics=['accuracy'])
multi_model.fit_generator(train_generator, steps_per_epoch=steps_train, epochs=2,
                   validation_data=valid_generator, validation_steps=steps_valid, max_queue_size=32, workers=4,
                   callbacks=[checkpoint, LR_reducer])

In [ ]:
#model.load_weights("/home/ubuntu/work/change_detect/program_redo_net_modify/weights/"+"inception_v3-v9-no-weight042516_0.9984.h5")              
multi_model.compile(optimizer=Adam(lr=1e-4), 
              loss='binary_crossentropy',
              metrics=['accuracy'])
multi_model.fit_generator(train_generator, steps_per_epoch=steps_train, epochs=1,
                   validation_data=valid_generator, validation_steps=steps_valid, max_queue_size=32, workers=4,
                   callbacks=[checkpoint, LR_reducer])

In [ ]:
            
multi_model.compile(optimizer=Adam(lr=1e-5), 
              loss='binary_crossentropy',
              metrics=['accuracy'])
multi_model.fit_generator(train_generator, steps_per_epoch=steps_train, epochs=1,
                   validation_data=valid_generator, validation_steps=steps_valid, max_queue_size=32, workers=4,
                   callbacks=[checkpoint,  LR_reducer])